# Assignment 2: Iterative-Deepening Search

*Maxwell You*

## Overview

In this notebook, I have implemented the iterative-deepening search (IDS) algorithm used for uninformed searches. For this assignment, I had to apply IDS to the 8-puzzle and a puzzle of my choice: the pegboard puzzle.

## Iterative Deepening and Depth Limited Search

IDS uses depth limited search (DLS) to search the game states. The signatures of the IDS algorithm are as follows:

  * `iterativeDeepeningSearch(startState, actionsF, takeActionF, goalState, goalTestF, maxDepth)`
  * `depthLimitedSearch(startState, actionsF, takeActionF, goalState, goalTestF, depthLimit)`
  
`depthLimitedSearch` is called by `iterativeDeepeningSearch` with `depthLimit`s of $0, 1, \ldots, $ `maxDepth`. Both must return either the solution path as a list of states, or the strings `cutoff` or `failure`.  `failure` signifies that all states were searched and the goal was not found. 

Each receives the arguments

  * the starting state, 
  * a function `actionsF` that is given a state and returns a list of valid actions from that state,
  * a function `takeActionF` that is given a state and an action and returns the new state that results from applying the action to the state,
  * the goal state,
  * a function `goalTestF` that is given a state and the goal state and returns `True` if the state satisfies the goal, and
  * either a `depthLimit` for `depthLimitedSearch`, or `maxDepth` for `iterativeDeepeningSearch`.

## Solving the 8-puzzle

The state of the puzzle is represented as a list of integers. 0 represents the empty position.

Functions for the 8-puzzle are the following:

  * `findBlank_8p(state)`: return the row and column index for the location of the blank (the 0 value).
  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. Actions are returned in the order `left`, `right`, `up`, `down`, though only if each one is a valid action.
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state`.
  * `printPath_8p(startState, goalState, path)`: print a solution path in a readable form.
  
**Disclaimer:** *When I say 'index' below, I mean the index of the number as it appears in the list.*

### Printing the state of the 8-puzzle

I traverse the state one row at a time and print the numbers with a trailing whitespace, unless it is the last number in the row, then I just print it. If a 0 is found, a space is printed instead.

In [47]:
def printState_8p(state):
    """
    Prints the state of the 8 puzzle as a 3x3 board given a state in the form of a list
    :param state:
    :return:
    """
    for i in range(3):
        if state[i] == 0:
            if i == 2:
                print(' ')
                break
            print('  ', end='') # print a space for 0 and a trailing space
        elif i == 2:
            print(state[i])
        else:
            print(state[i], end=' ')
    for i in range(3, 6):
        if state[i] == 0:
            if i == 5:
                print(' ')
                break
            print('  ', end='') # print a space for 0 and a trailing space
        elif i == 5:
            print(state[i])
        else:
            print(state[i], end=' ')
    for i in range(6, 9):
        if state[i] == 0:
            if i == 8:
                print(' ')
                break
            print('  ', end='') # print a space for 0 and a trailing space
        elif i == 8:
            print(state[i])
        else:
            print(state[i], end=' ')

### Find the blank

Finding which rows the numbers belonged to was simple. Finding the column they belonged in was a little trickier. Because this is a 3x3 puzzle, the only valid columns are `[0, 1, 2]`. The columns of the first three numbers (row 0) are just their respective indices (e.g. index 0 = column 0). 

To find the columns for numbers in indices 3-8, I first had to define the maximum index for each row. If a number was in row 1 (indices 3-5), their maximum index would be 5. If we subtract the index of the number from 5, we get an offset that we can subtract from the maximum column, 2, to find the column that number would occupy in the 8-puzzle.

Example:

    state = [1, 2, 3, 4, 0, 5, 6, 7, 8]
    
    Find blank
    
    Index of 0 = 4
    
    row = 1
    
    Maximum index for row 1 = 5
    
    col = 2 - (5 - 4)
    
    col = 1
    
    (row, col) of blank = (1, 1)
    
    1 2 3
    4 0 5   # Indeed, the blank is at (1, 1)
    6 7 8

In [48]:
def findBlank_8p(state):
    """
    Returns tuple with coordinates of blank
    Assumes that there will be a blank (0) in the input list
    :param state:
    :return (row, col):
    """
    row = -1
    col = -1
    for i in range(len(state)):
        if i < 3 and state[i] == 0:
                row = 0                # in 1st row if (0 <= index in list < 3)
                col = i                # col is the offset within first 3 indices
                break
        elif i < 6 and state[i] == 0:
                row = 1                # in 2nd row if (3 <= index in list < 6)
                col = 2 - (5 - i)      # col is the offset within 3rd-5th indices
                break
        elif i < 9 and state[i] == 0:
                row = 2                # in 3rd row if (6 <= index in list < 9)
                col = 2 - (8 - i)      # col is the offset within 6th-8th indices
                break
    return (row, col)

### List the valid actions

A blank can only move left, right, up, or down. Some manipulation of row and col allowed me to find the valid actions.

In [49]:
def actionsF_8p(state):
    """
    Returns list of valid actions based on the position of the blank
    Assumes a valid state with blank is provided
    :param state:
    :return actions:
    """
    actions = []
    row, col = findBlank_8p(state)
    if 0 <= col - 1 < 3:
        actions.append('left')
    if 0 <= col + 1 < 3:
        actions.append('right')
    if 0 <= row - 1 < 3:
        actions.append('up')
    if 0 <= row + 1 < 3:
        actions.append('down')

    return actions

### Executing a valid action

In order to write `takeActionF_8p`, I wrote a helper function to find the index of the blank given the row and column of the blank in the hypothetical 2D gameboard. The function is called `blankIndexInList`:

In [50]:
def blankIndexInList(row, col):
    """
    Returns index of blank in the list based on its row, col in the gameboard
    Assumes valid row, col are given
    :param row: 
    :param col: 
    :return index: 
    """
    index = -1
    if row == 0:
        index = col
    if row == 1:
        index = 3 + col
    if row == 2:
        index = 6 + col
    return index

Using this helper, `takeActionF_8p` was much easier to write as I discovered ways to manipulate the index of the blank given what action was to be performed. For example, if the index of the blank is 1, then an action of 'down' would move the blank to index 4. Before we do that though, we have to make sure we move the value that will be swapped with the blank into the blank's position (so we don't lose its value).

The `takeActionF_8p` function is defined below:

In [51]:
def takeActionF_8p(state, action):
    """
    Returns the modified state list by applying the action
    Assumes only valid actions are given
    :param state:
    :param action:
    :return state:
    """
    stateCopy = state[:]  # copy state into a new list, so we dont modify the original
    row, col = findBlank_8p(state)
    if action == 'up':
        moveFrom = blankIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = stateCopy[moveFrom - 3]  # swap blank with tile above
        stateCopy[moveFrom - 3] = 0  # place blank in tile above

    elif action == 'down':
        moveFrom = blankIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = stateCopy[moveFrom + 3]  # swap blank with tile below
        stateCopy[moveFrom + 3] = 0  # place blank in tile below

    elif action == 'left':
        moveFrom = blankIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = state[moveFrom - 1]  # swap blank with left tile
        stateCopy[moveFrom - 1] = 0  # place blank in left tile

    elif action == 'right':
        moveFrom = blankIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = stateCopy[moveFrom + 1]  # swap blank with right tile
        stateCopy[moveFrom + 1] = 0  # place blank in right tile
    return stateCopy  # return updated state

## Depth Limited Search

Depth limited search depth first searches the tree in a breadth-first-like manner. DLS runs DFS down a branch until the `depthLimit` is reached. When it reaches the `depthLimit`, it will backtrack to the last unexplored node and DFS down that branch to the `depthLimit`. When it begins to explore the new node, it no longer keeps the previous branch of nodes in memory, thereby providing the space benefit of DFS.

Depth limited search on its own is not complete nor optimal (i.e. not guaranteed to find a solution or find the best one).

It is not complete because choosing a depth limit *l* that is less than the depth *d* of the shallowest goal node (*l < d*) would lead it to find no solution.

It is not optimal because if we choose *l > d*, then it could find a goal node at a depth below *d* that would be correct, but not the shortest path.

In [52]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'

    cutoffOccurred = False

    for action in actionsF(state):
        childState = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit - 1)
        if result == 'cutoff':
            cutoffOccurred = True
        elif result is not 'failure':
            result.insert(0, childState)
            return result

    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'

## Iterative Deepening Search

Iterative deepening search improves upon DLS by trying incremental max depths. In doing so, this search is both complete and optimal.

It is complete, when the branching factor is finite, because it will search every level of the state space.

It is optimal because it DLSs every level, so if a goal node is found, then we know that this is the shortest path to the goal node. If there were a shorter path to this goal node, it would have been found in the previous level or before this node on the same level.

In [53]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            result.insert(0, startState)
            return result
    return 'cutoff'

### Printing the path

In [54]:
def printPath_8p(startState, goalState, path):
    print('Path from')
    printState_8p(startState)
    print('  to')
    printState_8p(goalState)
    print('is ' + str(len(path)) + ' nodes long:')
    print()
    for p in path:
        printState_8p(p)
        print()

Here are some example results.

In [55]:
startState = [1, 0, 3, 4, 2, 5, 6, 7, 8]

In [56]:
printState_8p(startState)

1   3
4 2 5
6 7 8


In [57]:
findBlank_8p(startState)

(0, 1)

In [58]:
actionsF_8p(startState)

['left', 'right', 'down']

In [59]:
takeActionF_8p(startState, 'down')

[1, 2, 3, 4, 0, 5, 6, 7, 8]

In [60]:
printState_8p(takeActionF_8p(startState, 'down'))

1 2 3
4   5
6 7 8


In [61]:
goalState = takeActionF_8p(startState, 'down')

In [62]:
newState = takeActionF_8p(startState, 'down')

In [63]:
newState == goalState

True

In [64]:
startState

[1, 0, 3, 4, 2, 5, 6, 7, 8]

In [65]:
path = depthLimitedSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

[[0, 1, 3, 4, 2, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

Notice that `depthLimitedSearch` result is missing the start state.  This is inserted by `iterativeDeepeningSearch`.

But, when we try `iterativeDeepeningSearch` to do the same search, it finds a shorter path!

In [66]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

[[1, 0, 3, 4, 2, 5, 6, 7, 8], [1, 2, 3, 4, 0, 5, 6, 7, 8]]

In [67]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

'cutoff'

In [68]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 5)
path

'cutoff'

Sometimes, we can't reach the goal state from just any start state. We need a way to randomly generate a valid start state. The idea here is kind of like scrambling a Rubik's Cube from the solved state into a jumbled state, then solving it from there.

In [69]:
import random

In [70]:
random.choice(['left', 'right'])

'left'

In [71]:
def randomStartState(goalState, actionsF, takeActionF, nSteps):
    state = goalState
    for i in range(nSteps):
        state = takeActionF(state, random.choice(actionsF(state)))
    return state

In [72]:
startState = randomStartState(goalState, actionsF_8p, takeActionF_8p, 10)
startState

[1, 2, 3, 4, 5, 8, 6, 7, 0]

In [73]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 20)
path

[[1, 2, 3, 4, 5, 8, 6, 7, 0],
 [1, 2, 3, 4, 5, 0, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

Let's print out the state sequence in a readable form.

In [74]:
for p in path:
    printState_8p(p)
    print()

1 2 3
4 5 8
6 7  

1 2 3
4 5  
6 7 8

1 2 3
4   5
6 7 8



Below, I use the `printPath_8p` function to print the solution path in a readable format.

In [75]:
printPath_8p(startState, goalState, path)

Path from
1 2 3
4 5 8
6 7  
  to
1 2 3
4   5
6 7 8
is 3 nodes long:

1 2 3
4 5 8
6 7  

1 2 3
4 5  
6 7 8

1 2 3
4   5
6 7 8



## Solving the Pegboard puzzle

The Pegboard puzzle seems to be most commonly found in Cracker Barrel restuarants. It is played on a 5x5 triangular gameboard populated with 14 pins and an empty hole. A peg can effectively 'jump' over another peg into an empty hole to 'eat' that peg. The objective of the game is to move the pegs so that only one peg remains at the end.

Example:

        0                     1
       1 1                   0 0
      1 1 1       -->       0 0 0
     1 1 1 1               0 0 0 0
    1 1 1 1 1             0 0 0 0 0
    
In order to solve this puzzle, I will be thinking in terms of how the blank(s) can move and not the pegs themselves. This is similar to the logic used to solve the 8-puzzle. It is also worth noting that there are only four starting positions as any other starting position is a rotation of one of those four. Also, there are only four ending states possible. These observations were taken from: [Pegboard Puzzle Solution Page](http://pegboardgame.blogspot.com)

### Helper functions

* `printState_pb(state)`: print the state of the board
* `printPath_pb(startState, goalState, path)`: print the solution path
* `findBlank_pb(state)`: return a list of coordinates of every blank on the board
* `coordToIndex(row, col)`: return an index into the state list based off the coordinates of the blank
* `actionsF_pb(state)`: returns a list of coordinate, action pairs that contain all the valid actions for those blanks
* `takeActionF_pb(state, action)`: returns the state of the board after taking the action

**Note:** *I refer to empty holes on the pegboard as 'blanks'*

### Coordinates of the board

This contains a list of valid coordinates on the board. It will be used to check boundaries `actionsF_pb`.

In [76]:
pb = [(0, 0),
      (1, 0), (1, 1),
      (2, 0), (2, 1), (2, 2),
      (3, 0), (3, 1), (3, 2), (3, 3),
      (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]

### Printing the state of the board

I padded the numbers to make the output look like a pegboard.

In [77]:
def printState_pb(state):
    print('    {0}'.format(state[0]))
    print('   {0} {1}'.format(state[1], state[2]))
    print('  {0} {1} {2}'.format(state[3], state[4], state[5]))
    print(' {0} {1} {2} {3}'.format(state[6], state[7], state[8], state[9]))
    print('{0} {1} {2} {3} {4}'.format(state[10], state[11], state[12], state[13], state[14]))

### Printing the solution path

Pretty much the same as the printPath of the 8-puzzle.

In [78]:
def printPath_pb(startState, goalState, path):
    # Assumes a valid solution path is provided (i.e. failure was not returned)
    
    print('Path from')
    printState_pb(startState)
    print('   to')
    printState_pb(goalState)
    print('is ' + str(len(path)) + ' nodes long:')
    print()
    for p in path:
        printState_pb(p)
        print()

### Find the blanks

Similar thought process as finding blanks for the 8-puzzle. The pegboard had more rows and columns to check for.

In [79]:
def findBlank_pb(state):
    """
    Return a list of coordinates (row, col) of every blank on the board
    :param state:
    :return blanks:
    """
    blanks = []

    # blanks are in a specific row depending on what range of indices blank is found in
    for i in range(len(state)):
        if i == 0 and state[i] == 0:    # blank in 0th row
            row = 0
            col = 0
            blanks.append((row, col))
            continue                    # look for next blank
        elif i < 3 and state[i] == 0:
            row = 1
            col = 1 - (2 - i)
            blanks.append((row, col))
            continue
        elif i < 6 and state[i] == 0:
            row = 2
            col = 2 - (5 - i)
            blanks.append((row, col))
            continue
        elif i < 10 and state[i] == 0:
            row = 3
            col = 3 - (9 - i)
            blanks.append((row, col))
            continue
        elif i < 15 and state[i] == 0:  # blank in 4th row
            row = 4
            col = 4 - (14 - i)
            blanks.append((row, col))
            continue
    return blanks

### Converting a coordinate to an index

The triangular nature of the board introduced some interesting patterns. For example, starting in row 0 and going down the left diagonal, indices would increase by 1, 2, 3, 4 depending on what row they were on. If you go down the left diagonal from row 2 to row 3, the indicies increase by 3 (e.g. index 4 --> index 7).

Picture:

           0
         1   2
       3   4  5
      6  7  8  9
    10 11 12 13 14

In [80]:
def coordToIndex(row, col):
    """
    Returns an index into the state list based off the coordinates of the blank
    Assumes a valid blank coordinate is given
    :param row:
    :param col:
    :return index:
    """

    index = -1              # location of blank in list

    if row == 0:
        index = 0
    elif row == 1:
        index = col + 1     # use column plus an offset to find index
    elif row == 2:
        index = col + 3
    elif row == 3:
        index = col + 6
    else:                   # row is 4
        index = col + 10
    return index

### Listing valid actions

The tricky part of running iterative deepening search on this puzzle was figuring out the valid actions.

In the pegboard puzzle, the start state will only has one blank that can move. But as the blank moves, it opens up more blanks to move. For example, going from the start state to the next leaves two free blanks to move in the next step.

With the 8-puzzle, I only had to record the valid actions of the single blank. With multiple blanks, **I had to record the valid moves for each blank.** This list of valid moves will be used in IDS to check what moves it should check next.

Because of this, I implemented the valid actions as (coordinate, action) pairs. I use `findBlanks_pb` to find the positions of all the blanks. Then, I loop through each blank and add its coordinates and any valid moves into a list containing valid actions.

In [81]:
def actionsF_pb(state):
    """
    Returns a list of coordinate, action pairs that contain all the valid actions for those blanks
    Assumes a valid state is given
    :param state:
    :return actions:
    """
    actions = []
    blanks = findBlank_pb(state)    # find coordinates of all blanks

    for blank in blanks:
        row, col = blank

        if row in [0, 1]:  # if blank is in 0th or 1st row
            if (row + 2, col) in pb and (state[coordToIndex(row + 1, col)] == 1
                                         and state[coordToIndex(row + 2, col)] == 1):
                actions.append((blank, 'l_diag_down'))
            if (row + 2, col + 2) in pb and (state[coordToIndex(row + 1, col + 1)] == 1
                                             and state[coordToIndex(row + 2, col + 2)] == 1):    # left diagonal down
                actions.append((blank, 'r_diag_down'))

        if row == 2:
            if (row + 2, col) in pb and (state[coordToIndex(row + 1, col)] == 1
                                         and state[coordToIndex(row + 2, col)] == 1):
                actions.append((blank, 'l_diag_down'))
            if (row + 2, col + 2) in pb and (state[coordToIndex(row + 1, col + 1)] == 1
                                             and state[coordToIndex(row + 2, col + 2)] == 1):    # left diagonal down
                actions.append((blank, 'r_diag_down'))
            if (row - 2, col) in pb and (state[coordToIndex(row - 1, col)] == 1
                                         and state[coordToIndex(row - 2, col)] == 1):
                actions.append((blank, 'l_diag_up'))
            if (row - 2, col - 2) in pb and (state[coordToIndex(row - 1, col - 1)] == 1
                                             and state[coordToIndex(row - 2, col - 2)] == 1):
                actions.append((blank, 'r_diag_up'))
            if (row, col + 2) in pb and (state[coordToIndex(row, col + 1)] == 1
                                         and state[coordToIndex(row, col + 2)] == 1):
                actions.append((blank, 'r_horiz'))
            if (row, col - 2) in pb and (state[coordToIndex(row, col - 1)] == 1
                                         and state[coordToIndex(row, col - 2)] == 1):
                actions.append((blank, 'l_horiz'))

        if row in [3, 4]:
            if (row - 2, col) in pb and (state[coordToIndex(row - 1, col)] == 1
                                         and state[coordToIndex(row - 2, col)] == 1):
                actions.append((blank, 'l_diag_up'))
            if (row - 2, col - 2) in pb and (state[coordToIndex(row - 1, col - 1)] == 1
                                             and state[coordToIndex(row - 2, col - 2)] == 1):
                actions.append((blank, 'r_diag_up'))
            if (row, col + 2) in pb and (state[coordToIndex(row, col + 1)] == 1
                                         and state[coordToIndex(row, col + 2)] == 1):
                actions.append((blank, 'r_horiz'))
            if (row, col - 2) in pb and (state[coordToIndex(row, col - 1)] == 1
                                         and state[coordToIndex(row, col - 2)] == 1):
                actions.append((blank, 'l_horiz'))

    return actions

### Executing a valid action

Executing an action is a matter of converting the coordinates into indices so that we can manipulate the state list accordingly.

In [82]:
def takeActionF_pb(state, action):
    """
    Returns the state of the board after taking the action
    action comes in as a tuple of coordinates (e.g. (3, 2)) and an action (e.g. l_diag_up)
    :param state:
    :param action:
    :return:
    """
    stateCopy = state[:]    # copy state into a new list, so we dont modify the original
    row, col = action[0]    # coordinates are the first part of action tuple
    move = action[1]        # move to be taken is second part of actions tuple

    if move == 'l_diag_down':
        stateCopy[coordToIndex(row + 1, col)] = 0
        stateCopy[coordToIndex(row + 2, col)] = 0
        stateCopy[coordToIndex(row, col)] = 1

    if move == 'r_diag_down':
        stateCopy[coordToIndex(row + 1, col + 1)] = 0
        stateCopy[coordToIndex(row + 2, col + 2)] = 0
        stateCopy[coordToIndex(row, col)] = 1

    if move == 'l_diag_up':
        stateCopy[coordToIndex(row - 1, col)] = 0
        stateCopy[coordToIndex(row - 2, col)] = 0
        stateCopy[coordToIndex(row, col)] = 1

    if move == 'r_diag_up':
        stateCopy[coordToIndex(row - 1, col - 1)] = 0
        stateCopy[coordToIndex(row - 2, col - 2)] = 0
        stateCopy[coordToIndex(row, col)] = 1

    if move == 'r_horiz':
        stateCopy[coordToIndex(row, col + 1)] = 0
        stateCopy[coordToIndex(row, col + 2)] = 0
        stateCopy[coordToIndex(row, col)] = 1

    if move == 'l_horiz':
        stateCopy[coordToIndex(row, col - 1)] = 0
        stateCopy[coordToIndex(row, col - 2)] = 0
        stateCopy[coordToIndex(row, col)] = 1

    return stateCopy

## Pegboard puzzle tests

In [83]:
startState = [0, 1, 1, 1, 1,
              1, 1, 1, 1, 1,
              1, 1, 1, 1, 1]
startState

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [84]:
printState_pb(startState)

    0
   1 1
  1 1 1
 1 1 1 1
1 1 1 1 1


In [85]:
print('blank(s) at: {0}'.format(findBlank_pb(startState)))

blank(s) at: [(0, 0)]


In [86]:
print('valid action(s): {0}'.format(actionsF_pb(startState)))

valid action(s): [((0, 0), 'l_diag_down'), ((0, 0), 'r_diag_down')]


In [87]:
print('moving blank (0, 0) to (2, 0):')
printState_pb(takeActionF_pb(startState, ((0, 0), 'l_diag_down')))

moving blank (0, 0) to (2, 0):
    1
   0 1
  0 1 1
 1 1 1 1
1 1 1 1 1


### Running some additional tests

In [88]:
test = [1, 1, 1, 0, 1,    # some random board state
        1, 0, 0, 1, 1,
        1, 1, 1, 0, 1]

In [89]:
print('blank(s) at: {0}'.format(findBlank_pb(test)))
print('valid action(s) for: ')
printState_pb(test)
print('{0}'.format(actionsF_pb(test)))

blank(s) at: [(2, 0), (3, 0), (3, 1), (4, 3)]
valid action(s) for: 
    1
   1 1
  0 1 1
 0 0 1 1
1 1 1 0 1
[((2, 0), 'l_diag_up'), ((2, 0), 'r_horiz'), ((3, 1), 'l_diag_up'), ((3, 1), 'r_horiz'), ((4, 3), 'r_diag_up'), ((4, 3), 'l_horiz')]


### Now to search for a solution to this pegboard

We are going to try to get from here:

In [90]:
printState_pb(startState)

    0
   1 1
  1 1 1
 1 1 1 1
1 1 1 1 1


to here:

In [91]:
goalState = [1, 0, 0, 0, 0,
             0, 0, 0, 0, 0,
             0, 0, 0, 0, 0]
printState_pb(goalState)

    1
   0 0
  0 0 0
 0 0 0 0
0 0 0 0 0


This takes a while! About a minute and a half.

In [92]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_pb, takeActionF_pb, 20)

In [93]:
printPath_pb(startState, goalState, path)

Path from
    0
   1 1
  1 1 1
 1 1 1 1
1 1 1 1 1
   to
    1
   0 0
  0 0 0
 0 0 0 0
0 0 0 0 0
is 14 nodes long:

    0
   1 1
  1 1 1
 1 1 1 1
1 1 1 1 1

    1
   0 1
  0 1 1
 1 1 1 1
1 1 1 1 1

    1
   0 1
  1 1 1
 1 0 1 1
1 1 0 1 1

    1
   1 1
  0 1 1
 0 0 1 1
1 1 0 1 1

    0
   0 1
  1 1 1
 0 0 1 1
1 1 0 1 1

    0
   0 0
  1 0 1
 0 1 1 1
1 1 0 1 1

    0
   0 1
  1 0 0
 0 1 1 0
1 1 0 1 1

    0
   0 1
  1 0 0
 0 1 1 0
1 1 1 0 0

    0
   0 1
  1 0 0
 0 1 1 0
1 0 0 1 0

    0
   0 1
  0 0 0
 0 0 1 0
1 0 1 1 0

    0
   0 1
  0 0 0
 0 0 1 0
1 1 0 0 0

    0
   0 1
  0 0 0
 0 0 1 0
0 0 1 0 0

    0
   0 1
  0 0 1
 0 0 0 0
0 0 0 0 0

    1
   0 0
  0 0 0
 0 0 0 0
0 0 0 0 0



Let's run another one!

In [94]:
gs2 = [0, 0, 0, 0, 0,
       0, 0, 0, 0, 0,
       0, 0, 1, 0, 0]

In [95]:
path = iterativeDeepeningSearch(startState, gs2, actionsF_pb, takeActionF_pb, 20)

In [96]:
printPath_pb(startState, gs2, path)

Path from
    0
   1 1
  1 1 1
 1 1 1 1
1 1 1 1 1
   to
    0
   0 0
  0 0 0
 0 0 0 0
0 0 1 0 0
is 14 nodes long:

    0
   1 1
  1 1 1
 1 1 1 1
1 1 1 1 1

    1
   0 1
  0 1 1
 1 1 1 1
1 1 1 1 1

    1
   1 1
  0 0 1
 1 1 0 1
1 1 1 1 1

    1
   1 1
  1 0 1
 0 1 0 1
0 1 1 1 1

    1
   1 1
  1 1 1
 0 0 0 1
0 0 1 1 1

    1
   0 1
  0 1 1
 1 0 0 1
0 0 1 1 1

    1
   0 0
  0 0 1
 1 1 0 1
0 0 1 1 1

    1
   0 1
  0 0 0
 1 1 0 0
0 0 1 1 1

    0
   0 0
  0 0 1
 1 1 0 0
0 0 1 1 1

    0
   0 0
  0 0 1
 0 0 1 0
0 0 1 1 1

    0
   0 0
  0 0 1
 0 0 1 0
0 1 0 0 1

    0
   0 0
  0 0 0
 0 0 0 0
0 1 1 0 1

    0
   0 0
  0 0 0
 0 0 0 0
0 0 0 1 1

    0
   0 0
  0 0 0
 0 0 0 0
0 0 1 0 0



## Reflection

It takes about about a minute to a few minutes for IDS to solve the pegboard puzzle depending on which start and goal states you define. Sometimes, you'll specify a goal state that can't be reached from that start state. I did not think of a better way to generate a start state from a goal state like was done above for the 8-puzzle. I ran an example from the blog post mentioned above that I knew was solvable. For another puzzle, I tried some other start and goal states and chose one that worked.

When IDS fails, it takes a while for it to figure out it failed. For example, the start state:
    
        1
       1 1
      0 1 1
     1 1 1 1
    1 1 1 1 1
    
and the goal state:

        0
       0 0
      0 0 0
     0 0 0 0
    0 0 1 0 0

took ~6 minutes to terminate and when it did, 'failure' was returned. I ran this with a `maxDepth` input of `20`. In order to find the solution, I would have to increase the maxDepth, but since it is already taking ~6 minutes to get to this point, I did not try.

I do not think iterative deepening search is the best algorithm to solve the pegboard puzzle. I believe it has to do with having multiple pegs that can move, and each peg having multiple moves as well. I think it leads to a really large branching factor that slows down the algorithm. While this is a problem that would affect most algorithms, I believe an informed search algorithm would probably solve this in faster time, or running a different uninformed search algorithm.

In [97]:
%run -i A2graderfinal.py

FileNotFoundError: [Errno 2] No such file or directory: 'nbconvert.py'